In [1]:
from __future__ import division
from __future__ import print_function

import time
import torch
import numpy as np
from numpy import argmax
import torch.nn.functional as F
from pygcn.gcnio.data import dataio
from pygcn.gcnio.util import utils
from pygcn.gcn3 import GCN
import scipy.sparse
import json
from sklearn.preprocessing import StandardScaler
import glog as log
import torch.optim as optim
print(torch.__version__)
#from torch.profiler import profile, record_function, ProfilerActivity

1.8.1+cu111


In [2]:
cuda = torch.cuda.is_available()
print('cuda: %s' % cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = 'cpu'

cuda: True


In [3]:
def load_data(prefix, normalize=True):
    adj_full = scipy.sparse.load_npz('./{}/adj_full.npz'.format(prefix))
    adj_train = scipy.sparse.load_npz('./{}/adj_train.npz'.format(prefix))
    role = json.load(open('./{}/role.json'.format(prefix)))
    feats = np.load('./{}/feats.npy'.format(prefix))
    class_map = json.load(open('./{}/class_map.json'.format(prefix)))
    class_map = {int(k):v for k,v in class_map.items()}
    assert len(class_map) == feats.shape[0]
    # ---- normalize feats ----
    train_nodes = np.array(list(set(adj_train.nonzero()[0])))
    train_feats = feats[train_nodes]
    scaler = StandardScaler()
    scaler.fit(train_feats)
    feats = scaler.transform(feats)
    # -------------------------
    return adj_full, adj_train, feats, class_map, role


def process_graph_data(adj_full, adj_train, feats, class_map, role, name):
    """
    setup vertex property map for output classes, train/val/test masks, and feats
    INPUT:
        G           graph-tool graph, full graph including training,val,testing
        feats       ndarray of shape |V|xf
        class_map   dictionary {vertex_id: class_id}
        val_nodes   index of validation nodes
        test_nodes  index of testing nodes
    OUTPUT:
        G           graph-tool graph unchanged
        role        array of size |V|, indicating 'train'/'val'/'test'
        class_arr   array of |V|x|C|, converted by class_map
        feats       array of features unchanged
    """
    num_vertices = adj_full.shape[0]
    if isinstance(list(class_map.values())[0],list):
        print("labels are list")
        num_classes = len(list(class_map.values())[0])
        class_arr = np.zeros((num_vertices, 1))
        p = 0;
        for k,v in class_map.items():
            class_arr[p] = argmax(v)
            p = p+1
    else:
        num_classes = max(class_map.values()) - min(class_map.values()) + 1
        class_arr = np.zeros((num_vertices, 1))
        for k,v in class_map.items():
            class_arr[k] = v
    if name=='flickr' or name=='reddit' or name=='ppi' or name=='amazon' or name=='yelp':
        class_arr = np.squeeze(class_arr.astype(int))
        
    return adj_full, adj_train, feats, class_arr, role

In [28]:
# make sure you use the same data splits as you generated attacks
seed = 15
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

# load original dataset (to get clean features and labels)
SMALL = False
if SMALL:
    dataset = 'cora'
    data = dataio.Dataset(root='/tmp/', name=dataset)
    adj, features, labels = data.adj, data.features, data.labels
    idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
    
    log.info(type(adj))
    log.info(adj.shape)
    log.info(type(features))
    log.info(features.shape)
    log.info(type(labels))
    log.info(labels.shape)
    log.info(type(idx_train))
    log.info(idx_train.shape)
    log.info(type(idx_val))
    log.info(idx_val.shape)
    log.info(type(idx_test))
    log.info(idx_test.shape)
else:
    data_prefix = './dataset/reddit'
    temp_data = load_data(data_prefix)
    data_list = data_prefix.split('/')
    print(data_list[-1])
    train_data = process_graph_data(*temp_data,data_list[-1])
    adj,adj_train,features,labels,role = train_data
    features = scipy.sparse.csr_matrix(features)
    idx_train = np.array(role['tr'])
    idx_val = np.array(role['va'])
    idx_test = np.array(role['te'])
    log.info(type(adj))
    log.info(adj.shape)
    log.info(type(adj_train))
    log.info(adj_train.shape)
    log.info(type(features))
    log.info(features.shape)
    log.info(type(labels))
    log.info(labels.shape)
    log.info(type(labels[0]))
    log.info(type(idx_train))
    log.info(idx_train.shape)
    log.info(type(idx_val))
    log.info(idx_val.shape)
    log.info(type(idx_test))
    log.info(idx_test.shape)

'''
flickr: (89250,1)
ppi:    (14755,121)
reddit: (232965,1)
amazon: (1569960,107)
yelp:   (716847,100)
'''

reddit


I0319 22:02:42.262606 4916 <ipython-input-28-5afab1ff2513>:39] <class 'scipy.sparse.csr.csr_matrix'>
I0319 22:02:42.263366 4916 <ipython-input-28-5afab1ff2513>:40] (232965, 232965)
I0319 22:02:42.263874 4916 <ipython-input-28-5afab1ff2513>:41] <class 'scipy.sparse.csr.csr_matrix'>
I0319 22:02:42.264352 4916 <ipython-input-28-5afab1ff2513>:42] (232965, 232965)
I0319 22:02:42.264815 4916 <ipython-input-28-5afab1ff2513>:43] <class 'scipy.sparse.csr.csr_matrix'>
I0319 22:02:42.265388 4916 <ipython-input-28-5afab1ff2513>:44] (232965, 602)
I0319 22:02:42.265848 4916 <ipython-input-28-5afab1ff2513>:45] <class 'numpy.ndarray'>
I0319 22:02:42.266308 4916 <ipython-input-28-5afab1ff2513>:46] (232965,)
I0319 22:02:42.266760 4916 <ipython-input-28-5afab1ff2513>:47] <class 'numpy.int64'>
I0319 22:02:42.267226 4916 <ipython-input-28-5afab1ff2513>:48] <class 'numpy.ndarray'>
I0319 22:02:42.267676 4916 <ipython-input-28-5afab1ff2513>:49] (153932,)
I0319 22:02:42.268214 4916 <ipython-input-28-5afab1ff25

'\nflickr: (89250,1)\nppi:    (14755,121)\nreddit: (232965,1)\namazon: (1569960,107)\nyelp:   (716847,100)\n'

In [29]:
print(labels[0])
print(labels[1])
print(labels[8])
print(labels.max())

model = GCN(nfeat=features.shape[1], nhid=32, nclass=labels.max()+1, device=device)
  
optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)

30
17
38
40


In [30]:
model = model.to(device)
TRAIN = 1
if TRAIN:
    #with profile(activities=[
    #    ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    #    with record_function("model_fit"):
    #        model.fit(features, adj, labels, idx_train, train_iters=200, verbose=True, name='ppi')
    #        torch.save(model.state_dict(),'./model/gcn.pt')
    #print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
    #prof.export_chrome_trace("trace.json")
    model.fit(features, adj, labels, idx_train, train_iters=200, verbose=True, name='reddit')
    torch.save(model.state_dict(),'./model/gcn.pt')
TEST = 1
if TEST:
    model.load_state_dict(torch.load('./model/gcn.pt'))
    model.eval()
    model.test(idx_test)

Transform data to GPU device ...


I0319 22:03:03.536324 4916 gcn3.py:181] torch.float32
I0319 22:03:03.537249 4916 gcn3.py:182] torch.float32


_train_without_val
Epoch 0, training loss: 3.7913706302642822
Epoch 10, training loss: 1.6687209606170654
Epoch 20, training loss: 0.8948564529418945
Epoch 30, training loss: 0.6476398706436157
Epoch 40, training loss: 0.5512953996658325
Epoch 50, training loss: 0.49943703413009644
Epoch 60, training loss: 0.4650373160839081
Epoch 70, training loss: 0.44400444626808167
Epoch 80, training loss: 0.4302442669868469
Epoch 90, training loss: 0.41057413816452026
Epoch 100, training loss: 0.4032043218612671
Epoch 110, training loss: 0.3940287232398987
Epoch 120, training loss: 0.38555556535720825
Epoch 130, training loss: 0.3775011897087097
Epoch 140, training loss: 0.3719058930873871
Epoch 150, training loss: 0.3623087704181671
Epoch 160, training loss: 0.35698258876800537
Epoch 170, training loss: 0.35585424304008484
Epoch 180, training loss: 0.34975022077560425
Epoch 190, training loss: 0.3454720675945282
Forward time: 154.8847s
Layer1 time: 0.0199s
Layer1 (AX)W time: 0.0103s
Layer reLU ti